# 2. Processar Dados de Consultas

Este notebook processa o relatório de "Slip Fatura" (Consultas) da operadora e atualiza a planilha de dados.

**Entradas necessárias:**
- Arquivo `dados_atualizados.xlsx` (gerado no passo anterior de Mensalidades)
- Relatório de Slip Fatura (Consultas) da operadora (arquivo `.xls`)

**Saída:**
- Arquivo `dados_com_consultas.xlsx` (Planilha final com Mensalidades e Consultas atualizadas)

In [ ]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

### 2.1 Carregamento dos Dados
Carrega o arquivo gerado anteriormente e o novo relatório de consultas.

**Atenção:** Configure o caminho do relatório de Slip Fatura abaixo.

In [ ]:
caminho_consultas = '' # Caminho para o arquivo de consultas | ../Data/caminho_consultas.xls ou ../Data/caminho_consultas.xlsx
caminho_dados = '' # Caminho para a planilha de dados | ../Data/dados.xlsx ou ../Data/dados_atualizados.xlsx
mes = '' # Mês Atual (Ex: 'JAN 2025')

dados = pd.read_excel(caminho_dados, skiprows=1 ,index_col=0, sheet_name=mes)
consultas = pd.read_excel(caminho_consultas, index_col=False)

### 2.2 Visualização da Estrutura
Verifica as colunas para garantir que o arquivo foi lido corretamente.

In [ ]:
display(dados.head())
display(consultas.head())

### 2.3 Processamento do Relatório de Consultas
O relatório de consultas possui uma estrutura hierárquica. O código abaixo varre o arquivo identificando:
1. Linhas que iniciam um bloco de beneficiário (`Evento` começa com "Beneficiário:")
2. Linhas que contêm o total das consultas (`Doc. Finan.` é "Total Crédito")
3. Linhas que encerram o bloco do beneficiário (`Doc. Finan.` é "Total Débito")

Extraímos o nome e o valor total de consultas para cada pessoa.

In [ ]:
def normaliza_codigo(codigo):
    if codigo is None or pd.isna(codigo):
        return pd.NA
    if isinstance(codigo, (int, float)):
        try:
            return str(int(codigo))
        except (ValueError, TypeError):
            return pd.NA
    texto = str(codigo).strip()
    if texto == '':
        return pd.NA
    if '.' in texto:
        texto = texto.split('.')[0]
    digits = ''.join(ch for ch in texto if ch.isdigit())
    return digits if digits != '' else pd.NA

def normaliza_valor(valor):
    if valor is None or pd.isna(valor):
        return pd.NA
    texto = str(valor).strip()
    if texto == '':
        return pd.NA

    if ',' in texto and '.' in texto:
        texto = texto.replace('.', '').replace(',', '.')
    elif ',' in texto:
        texto = texto.replace(',', '.')
    try:
        return float(texto)
    except ValueError:
        return pd.NA

result_totais = []

beneficiario_name = None
beneficiario_codigo = None

for index, row in consultas.iterrows():
    evento = row['Evento']
    doc_finan = row['Doc. Finan.']

    if evento is not None and isinstance(evento, str) and evento.startswith('Beneficiário:'):
        payload = evento.replace('Beneficiário:', '', 1).strip()
        partes = payload.split(' - ', 1)

        if len(partes) >= 2:
            beneficiario_codigo = partes[0].strip()
            nome_parte = partes[1]
        else:
            beneficiario_codigo = None
            nome_parte = payload

        try:
            beneficiario_name = nome_parte.split(' (')[0].strip()
        except Exception:
            beneficiario_name = nome_parte.strip()

    if doc_finan == 'Total Crédito' and beneficiario_name is not None:
        total_credito = normaliza_valor(row['Contrato Financeiro'])

        result_totais.append({
            'Codigo Beneficiário': beneficiario_codigo,
            'Nome Beneficiário': beneficiario_name,
            'Total Crédito': total_credito
        })

    if doc_finan == 'Total Débito' and beneficiario_name is not None:
        beneficiario_name = None
        beneficiario_codigo = None


beneficiarios_totais = pd.DataFrame(result_totais)
beneficiarios_totais['Codigo Beneficiário'] = beneficiarios_totais['Codigo Beneficiário'].apply(normaliza_codigo)

beneficiarios_agg = (
    beneficiarios_totais
    .dropna(subset=['Codigo Beneficiário'])
    .groupby('Codigo Beneficiário', as_index=False)
    .agg({'Nome Beneficiário': 'first', 'Total Crédito': 'max'})
)

print(f"Total (Total Crédito): R$ {beneficiarios_agg['Total Crédito'].sum():.2f}")
print(f"Total de beneficiários com consultas encontrados: {len(beneficiarios_agg)}")
display(beneficiarios_agg.head())

### 2.4 Validação de Totais (Opcional)
Exibe o valor total das consultas para conferência.

In [ ]:
total_geral = beneficiarios_agg['Total Crédito'].sum()
print(f"Valor total das consultas processadas: R$ {total_geral:.2f}")

### 2.4 Atualização da Planilha Principal
Transfere os valores encontrados para a coluna `CONSULTA` da planilha de dados.

In [ ]:
dados_carteira = dados['CARTEIRA'].apply(normaliza_codigo)
dados_nomes = dados['NOME'].astype('string').str.replace(r"\s+", " ", regex=True).str.strip()

nomes_nao_encontrados = []
nomes_multiplos = []

for index, row in beneficiarios_agg.iterrows():
    codigo = row['Codigo Beneficiário']
    beneficiario_name = str(row['Nome Beneficiário']).strip() if row['Nome Beneficiário'] is not None else None
    total_credito = row['Total Crédito']


    if codigo and codigo in dados_carteira.values:
        dados.loc[dados_carteira == codigo, 'CONSULTA'] = total_credito
    else:

        if beneficiario_name and beneficiario_name in dados_nomes.values:
            matches = dados_nomes == beneficiario_name
            if matches.sum() == 1:
                dados.loc[matches, 'CONSULTA'] = total_credito
            else:
                nomes_multiplos.append(beneficiario_name)
        elif beneficiario_name:
            nomes_nao_encontrados.append(beneficiario_name)

if nomes_multiplos:
    print("ATENÇÃO: nomes repetidos em dados (não atualizados por nome):")
    for nome in sorted(set(nomes_multiplos)):
        print(f"- {nome}")

if nomes_nao_encontrados:
    print("ATENÇÃO: nomes não encontrados em dados:")
    for nome in sorted(set(nomes_nao_encontrados)):
        print(f"- {nome}")

display(dados.head())

### 2.6 Relatório de Nomes Não Encontrados
Lista beneficiários que tiveram consulta mas não foram achados na planilha principal.

In [ ]:
if nomes_nao_encontrados:
    print("ATENÇÃO: Os seguintes nomes do relatório de consultas NÃO foram encontrados na planilha base:")
    for nome in nomes_nao_encontrados:
        print(f"- {nome}")
else:
    print("Sucesso: Todos os beneficiários com consultas foram encontrados e atualizados.")

### 2.7 Exportação Final
Salva o arquivo completo.

In [ ]:
dados.to_excel('../Data/dados_com_consultas.xlsx', index=False)
print('Arquivo salvo como "dados_com_consultas.xlsx" na pasta "Data".')